## memory_profiler

In [1]:
from datetime import date, timedelta
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm, tnrange

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

import mlflow
import mlflow.sklearn

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR,
    LAG_DICT,
    SLIDING_DICT
)

In [2]:
# solve lightgbm error on MAC
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# load data
df_train = pd.read_csv(
    RAW_DATA_DIR+'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    RAW_DATA_DIR+'test.csv', usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    RAW_DATA_DIR+'items.csv',
).set_index("item_nbr")

stores = pd.read_csv(
    RAW_DATA_DIR+'stores.csv',
).set_index("store_nbr")

### Test Period

2017-08-16 to 2017-08-31

In [4]:
test_start = date(2017, 8, 16)
test_end = date(2017,8, 31)

In [5]:
valid_start = test_start - timedelta(16)
while(1):
    if valid_start.weekday() == test_start.weekday():
        break
    valid_start = valid_start-timedelta(days=1)
valid_end = valid_start + timedelta(15)
print('valid starts from {} to {}'.format(valid_start, valid_end))

valid starts from 2017-07-26 to 2017-08-10


### Valid Period

Considering the more nearer peiods of sales data may have more in common, it would be better to find the nearest period as valid period.

Based on the analysis before, we assume the sales data is periodically with the frequency of 7 days, so we want to keep that feature same
in the train, valid and test period.

So finally, we choose valid period:

2017-07-26 to 2017-08-10


In [6]:
valid_start = date(2017, 7, 26)
valid_end = date(2017, 8, 10)

### Filter Period

#### Earthquake happended on April 16, 2016. It may affect for the next several weeks.

In [7]:
# filter the period which is affected by earthquake.
filter_date = date(2016,4,16) + timedelta(7*4)
lag_max = 140
train_start=  filter_date+timedelta(days=lag_max)

while(1):
    train_start = train_start + timedelta(1)
    if train_start.weekday() == valid_start.weekday():
        break
print('train datasets starts from {}'.format(train_start))

train datasets starts from 2016-10-05


In [8]:
train_start = date(2017, 4, 5)

### Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.


In [9]:
df_train = df_train[df_train['date']>=filter_date]

/home/y/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


#### Promo feature

In [10]:
promo_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

# missing onpromotions filling
promo_train = promo_train.unstack(level=-1).fillna(False)
promo_train.columns = promo_train.columns.get_level_values(1)

In [11]:
# missing test onpromotions filling
promo_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_test.columns = promo_test.columns.get_level_values(1)
# filter those items/stores in promo_test but not in promo_train
promo_test = promo_test.reindex(promo_train.index).fillna(False)

In [12]:
promo_features = pd.concat([promo_train, promo_test], axis=1)
del promo_test, promo_train

## Label

In [13]:
# label
df_train = df_train.set_index(["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(level=-1).fillna(0)

### Item

In [14]:
items = items.reindex(df_train.index.get_level_values(1))

#### Item Family

In [15]:
items['family'] = items['family'].astype('category')
item_family_features = items.family.cat.codes.values

#### Item's class

In [16]:
items['class'] = items['class'].astype('category')
item_class_features = items['class'].cat.codes.values

### Store

In [17]:
stores = stores.reindex(df_train.index.get_level_values(0))

#### Store's city

In [18]:
stores['city'] = stores['city'].astype('category')
store_city_features = stores['city'].cat.codes.values

#### Store's state

In [19]:
stores['state'] = stores['state'].astype('category')
store_state_features = stores['state'].cat.codes.values

#### Store's type

In [20]:
stores['type'] = stores['type'].astype('category')
store_type_features = stores['type'].cat.codes.values

#### Store's cluster

In [21]:
stores['cluster'] = stores['cluster'].astype('category')
store_cluster_features = stores['cluster'].cat.codes.values

In [22]:
df_train.columns = df_train.columns.get_level_values(1)

#### Filling missing date

In [23]:
date_list = df_train.columns
obj_list = pd.date_range(filter_date, test_start-timedelta(1))
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    df_train[i] = 0

2016-12-25 00:00:00


In [24]:
date_list = promo_features.columns
obj_list = pd.date_range(filter_date, test_end)
diff_list = list(set(obj_list) - set(date_list)) 
for i in diff_list:
    print(i)
    promo_features[i] = 0

2016-12-25 00:00:00


#### Lagging and sliding windows

In [25]:
LAG_DICT = {'unit_sales': [1,2,3, 4, 5, 6, 7, 8, 9, 10 ,11 ,12, 13 ,14, 15, 16, 21,30, 60],
            'onpromotion': [2, 3,4,5,6, 7, 14, 21]}

SLIDING_DICT = {'unit_sales': [3, 4, 5, 6, 7, 14, 21, 30, 60]}

# initialise dirs
RAW_DATA_DIR = 'datasets/'

In [26]:
train_start + timedelta(days=16)

datetime.date(2017, 4, 21)

In [27]:
def get_timespan(df, 
                 start_time,
                 minus,
                 periods,
                 freq='D'):
    return df[pd.date_range(start_time - timedelta(days=minus), periods=periods, freq=freq)]

def gen_dataset(df, 
                promo_features,
                item_family_features,
                item_class_features,
                store_city_features,
                store_state_features,
                store_type_features,
                store_cluster_features,
                start_time,
                is_train=True):
    # init
    X = pd.DataFrame()
    
    for i in LAG_DICT['unit_sales']:
        X['lag_{}_sales'.format(i)] = get_timespan(df, start_time, i, 1).values.ravel()
    
    for i in LAG_DICT['onpromotion']:
        X['sum_{}_promo'.format(i)] = get_timespan(promo_features, start_time, i, 1).sum(axis=1).ravel()
        
    for i in range(1, 16):
        X['sum_{}_promo_test'.format(i)]= get_timespan(promo_features, start_time + timedelta(days=16), 15, i).sum(axis=1).values
        
    for i in SLIDING_DICT['unit_sales']:
        X["mean_{}_sales".format(i)] = get_timespan(df, start_time, i, i).mean(axis=1).values
        X["std_{}_sales".format(i)] = get_timespan(df, start_time, i, i).std(axis=1).values
        X["min_{}_sales".format(i)] = get_timespan(df, start_time, i, i).min(axis=1).values
        X["max_{}_sales".format(i)] = get_timespan(df, start_time, i, i).max(axis=1).values
        X["median_{}_sales".format(i)] = get_timespan(df, start_time, i, i).median(axis=1).values


    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, start_time, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, start_time, 140-i, 20, freq='7D').mean(axis=1).values
        
    # for the next to-predict 16 days 
    for i in range(16):
        X["promo_{}".format(i)] = promo_features[start_time + timedelta(days=i)].values.astype(np.uint8)

    for i in [7, 14, 30, 60, 140]:
        tmp = get_timespan(df, start_time, i, i)

        X['has_sales_days_in_last_{}'.format(i)] = (tmp > 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values

        tmp = get_timespan(promo_features, start_time, i, i)
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values


        
#     X['item_family_features'] = item_family_features

#     X['item_class_features'] = item_class_features

#     X['store_city_features'] = store_city_features

#     X['store_state_features'] = store_state_features

#     X['store_type_features'] = store_type_features

#     X['store_cluster_features'] = store_cluster_features
        
    if is_train:
        y = df[pd.date_range(start_time, periods=16)].values
        return X, y
    return X


#### Generate train, valid and test sets

In [28]:
print("Preparing dataset...")

nbr_weeks = int((valid_start - train_start).days/7)

X_l, y_l = [], []

for i in tqdm(range(nbr_weeks), desc = 'No. of week'):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = gen_dataset(
        df_train,
        promo_features,
        item_family_features,
        item_class_features,
        store_city_features,
        store_state_features,
        store_type_features,
        store_cluster_features,
        train_start + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
#     break

No. of week:   0%|          | 0/16 [00:00<?, ?it/s]

Preparing dataset...


No. of week: 100%|██████████| 16/16 [01:04<00:00,  4.04s/it]


In [29]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

In [30]:

X_val, y_val = gen_dataset(df_train,
                           promo_features,
                           item_family_features,
                           item_class_features,
                           store_city_features,
                           store_state_features,
                           store_type_features,
                           store_cluster_features,
                           valid_start)
X_test = gen_dataset(df_train, 
                    promo_features,
                    item_family_features,
                    item_class_features,
                    store_city_features,
                    store_state_features,
                    store_type_features,
                    store_cluster_features,
                    test_start, is_train=False)

#### Train Model

In [31]:
import pandas as pd
import numpy as np

from config import (
    RAW_DATA_DIR,
    FEATURE_DIR
)

from keras.models import Model
from keras.layers import Input, Conv1D, Dense, Activation, Dropout, Lambda, Multiply, Add, Concatenate
from keras.optimizers import Adam

Using TensorFlow backend.


In [34]:
def wavenet_enhance_model(input_length, feature_length, output_length):
    # convolutional operation parameters
    n_filters = 32 # 32 
    filter_width = 2
    dilation_rates = [2**i for i in range(8)] * 2 

    # define an input history series and pass it through a stack of dilated causal convolution blocks. 
    history_seq = Input(shape=(input_length, feature_length))
    x = history_seq

    skips = []
    for dilation_rate in dilation_rates:

        # preprocessing - equivalent to time-distributed dense
        x = Conv1D(64, 1, padding='same', activation='relu')(x) 

        # filter convolution
        x_f = Conv1D(filters=n_filters,
                     kernel_size=filter_width, 
                     padding='causal',
                     dilation_rate=dilation_rate)(x)

        # gating convolution
        x_g = Conv1D(filters=n_filters,
                     kernel_size=filter_width, 
                     padding='causal',
                     dilation_rate=dilation_rate)(x)

        # multiply filter and gating branches
        z = Multiply()([Activation('tanh')(x_f),
                        Activation('sigmoid')(x_g)])

        # postprocessing - equivalent to time-distributed dense
        z = Conv1D(output_length, 1, padding='same', activation='relu')(z)

        # residual connection
        x = Add()([x, z])    

        # collect skip connections
        skips.append(z)

    # add all skip connection outputs 
    out = Activation('relu')(Add()(skips))

    # final time-distributed dense layers 
    out = Conv1D(128, 1, padding='same')(out)
    out = Activation('relu')(out)
    out = Dropout(.2)(out)
    out = Conv1D(output_length, 1, padding='same')(out)

    # extract the last time steps as the training target
    def slice(x, seq_length):
        return x[:,-seq_length:,:]

    pred_seq_train = Lambda(slice, arguments={'seq_length':output_length})(out)

    model = Model(history_seq, pred_seq_train)
    model.compile(Adam(), loss='mean_squared_error')
    return model

In [35]:
model = wavenet_enhance_model(1, X_train.shape[1], y_train.shape[1])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1, 137)       0                                            
__________________________________________________________________________________________________
conv1d_67 (Conv1D)              (None, 1, 16)        2208        input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_68 (Conv1D)              (None, 1, 32)        1056        conv1d_67[0][0]                  
__________________________________________________________________________________________________
conv1d_69 (Conv1D)              (None, 1, 32)        1056        conv1d_67[0][0]                  
____________________________________________________________________________________________

In [41]:
# x_scaler = StandardScaler()
# x_scaler.fit(X_train)
# X_train = x_scaler.transform(X_train)
# X_val = x_scaler.transform(X_val)
# X_test = x_scaler.transform(X_test)

# y_scaler = StandardScaler()
# y_scaler.fit(y_train)
# y_train = y_scaler.transform(y_train)
# y_val = y_scaler.transform(y_val)

x = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
y = np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1]))

x_v = np.reshape(X_val.values, (X_val.shape[0], 1, X_val.shape[1]))
y_v = np.reshape(y_val, (y_val.shape[0], 1, y_val.shape[1]))

x_t = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

In [42]:
batch_size = 2**12
epochs = 10
model.compile(Adam(), loss='mean_squared_error')
history = model.fit(x,
                    y,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data = (x_v, y_v))


Train on 2742752 samples, validate on 171422 samples
Epoch 1/10
2742752/2742752 [==============================] - 769s 280us/step - loss: 0.4708 - val_loss: 0.3887
Epoch 2/10
2742752/2742752 [==============================] - 725s 264us/step - loss: 0.3966 - val_loss: 0.3874
Epoch 3/10
2742752/2742752 [==============================] - 729s 266us/step - loss: 0.3901 - val_loss: 0.3767
Epoch 4/10
2742752/2742752 [==============================] - 738s 269us/step - loss: 0.3865 - val_loss: 0.3746
Epoch 5/10
2742752/2742752 [==============================] - 724s 264us/step - loss: 0.3835 - val_loss: 0.3761
Epoch 6/10
2742752/2742752 [==============================] - 740s 270us/step - loss: 0.3813 - val_loss: 0.3742
Epoch 7/10
2742752/2742752 [==============================] - 723s 264us/step - loss: 0.3789 - val_loss: 0.3738
Epoch 8/10
2742752/2742752 [==============================] - 732s 267us/step - loss: 0.3770 - val_loss: 0.3676
Epoch 9/10
2742752/2742752 [======================

It's typically a good idea to look at the convergence curve of train/validation loss.

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
pred = model.predict(x)

In [ ]:
pred = model.predict(x_v)
y_v = np.reshape(y_v, (y_v.shape[0], y_v.shape[2]))
pred = np.reshape(pred, (pred.shape[0], pred.shape[2]))
mean_squared_error(y_v, pred)

In [ ]:
pred = model.predict(x)
y = np.reshape(y, (y.shape[0], y.shape[2]))
pred = np.reshape(pred, (pred.shape[0], pred.shape[2]))
mean_squared_error(y, pred)

In [ ]:
weight = items["perishable"] * 0.25 + 1
err = (y_val - np.array(val_pred).transpose())**2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print('nwrmsle = {}'.format(err)) #nwrmsle